## Setup
Set up some imports that we need for recipes

In [1]:
import {
  derive,
  handler,
  JSONSchema,
  NAME,
  UI,
  recipe,
  Schema,
  cell,
  run,
  getDoc
} from "../../packages/cli/notebook_shim.ts";


## Most simple recipe
The entry point is to call "recipe" which returns an anonymous function.
You also pass it a function that is what your recipe "does". It takes in a list of input references, does work, and then outputs a result.
We'll go into all that but for now, we take in nothing, and give back nothing. 
You can see that this function object has an `argumentSchema` (which is empty) and also a `resultSchema` which is also empty.


In [2]:
recipe("super simple recipe", () => ({}));

[Function (anonymous)] {
  argumentSchema: {
    type: "object",
    properties: {},
    description: "super simple recipe"
  },
  resultSchema: {},
  initial: {},
  result: {},
  nodes: [],
  toJSON: [Function: toJSON]
}

## UI conventions
Most recipes will have a [NAME] and a [UI] property. We use this when we render the recipes.
You can see them under `result`

In [3]:
const helloRecipe = recipe("super simple recipe", () => ({
  [NAME]: "Hello World Recipe",
  [UI]: {
    tag: "os-container",
    children: ["Hello World!"]
  }
}));
helloRecipe

[Function (anonymous)] {
  argumentSchema: {
    type: "object",
    properties: {},
    description: "super simple recipe"
  },
  resultSchema: {},
  initial: {},
  result: {
    "$NAME": "Hello World Recipe",
    "$UI": { tag: "os-container", children: [ "Hello World!" ] }
  },
  nodes: [],
  toJSON: [Function: toJSON]
}

## ArgumentSchema and ResultSchema
Now let's add an ArgumentSchema that contains a `name` property. 

In [4]:
const ArgumentSchema = {
  type: "object",
  properties: {
    name: { type: "string" }
  },
  required: ["name"]
} as const satisfies JSONSchema;

const ResultSchema = {
  type: "object",
  properties: {
    echoed: { type: "string" }
  },
  required: ["echoed"]
} as const satisfies JSONSchema;

const helloRecipe = recipe(ArgumentSchema, ResultSchema, ({ name }) => {
  // Create a derived value that reacts to changes in `name`
  const echoed = derive(name, (nameValue) => {
    console.log("name changed", nameValue);
    return `Hello, ${nameValue}!`;
  });

  return {
    [NAME]: "Hello World (with derive)",
    echoed // 👈 included in `result` per ResultSchema
  };
});

// make input a proper cell:
const input = getDoc("Ellyse", "name", "test").asCell();
const resultDoc = getDoc(undefined, "run hello", "test");
const output = await run(helloRecipe, { name: input }, resultDoc);
output

generateRecipeId: generated id ba4jcau2dsvfxx74m73e6w2bg37f7xlukxhann3zmww3qf7yq4sjlbbg3
name changed Ellyse


{
  get: [Function: get],
  getAsQueryResult: [Function: getAsQueryResult],
  asCell: [Function: asCell],
  send: [Function: send],
  updates: [Function: updates],
  getAtPath: [Function: getAtPath],
  setAtPath: [Function: setAtPath],
  freeze: [Function: freeze],
  isFrozen: [Function: isFrozen],
  toJSON: [Function: toJSON],
  value: [Getter],
  entityId: [Getter/Setter],
  space: [Getter/Setter],
  sourceCell: [Getter/Setter],
  ephemeral: [Getter/Setter],
  copyTrap: [Getter],
  [Symbol(toOpaqueRef)]: [Function: [toOpaqueRef]],
  [Symbol(isDoc)]: true
}